## Projet de deep learning réalisé dans le cadre du module d'IA.

### Sujet choisi : Classification d'images de chats et de chiens en utilisant un réseau neuronal convolutif

#### 1 - Import des librairies nécessaires au projet
- numpy : Utilisé pour gérer les tableaux de données
- pandas : Utilisé pour convertir les données de résultat en csv
- cv2 : Utilisé pour lire et traiter les images
- tensorflow : Utilisé pour créer notre réseau de neurone et l'entraîner

In [19]:
import numpy as np
import pandas as pd
import cv2
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D
import os

#### 2 - Récupération et pré-traitement des données

Ici, on crée une fonction qui parcourt toutes les images d'entraînement (dossier "dataset/train/) et plusieurs traitements sont effectués :
- On utilise cv2 pour lire l'image en niveau de gris et la transformer en tableau de pixels (0 à 255)
- Redimensionne le tableau avec un taille de 80 x 80
- Ajoute ce tableau au tableau de données d'entraînement "X".
- Ajoute la catégorie réelle, chien ou chat de l'image au tableau des résultats attendus "y". 

In [23]:
train_path = "./dataset/train/"

convert = lambda category : int(category == 'dog')
def create_test_data(train_path):
    X = []
    y = []
    for p in os.listdir(train_path):
        category = p.split(".")[0]
        category = convert(category)
        img_array = cv2.imread(os.path.join(train_path,p),cv2.IMREAD_GRAYSCALE)
        new_img_array = cv2.resize(img_array, dsize=(80, 80))
        X.append(new_img_array)
        y.append(category)
    return X, y


#### 3 - Création des données d'entraînements et normalisation
Ici nous appellons la fonction définie précédemment pour créer les données d'entrainements.
Avec des valeurs allant de 0 à 255, notre modèle est un peu moins lisible et général. C'est pourquoi nous normalisons ces valeurs afin de les ramener entre 0 et 1.

In [ ]:
X, y = create_test_data(train_path)
def reshape_and_normalize_data(X,y):
    X = np.array(X).reshape(-1, 80,80,1)
    y = np.array(y)
    X = X/255.0
    return X, y
X, y = reshape_and_normalize_data(X, y)

#### 4 - Création du modèle
Nous créons un modèle séquentiel à deux couches avec les caractéristiques suivantes :
- 64 filtres
- Un motif de taille (3,3) --> motif de kernel
- Une fonction d'activation relu permettant de remplacer les résultats négatifs par 0

Ces deux couches sont entièrement connectées, c'est-à-dire que chaque neurone est relié à chaque neurone de la couche précédente.

In [20]:
model = Sequential()
model.add(Conv2D(64,(3,3), activation = 'relu', input_shape = X.shape[1:]))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64,(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])

#### 5 - Entraînment du modèle
- Nous avons choisi d'entraîner notre modèle sur 10 époques. Cet à dire qu'on passe toutes les images d'entraînements 10 fois.
- On décide de passer 32 images par 32 pour réaliser une époque.
- On décide de prendre les derniers 20% des images d'entraînements pour tester notre modèle sur l'époque actuelle.

In [ ]:
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

#### 6 - Interprétation des résultats

Une fois le modèle entraîné, nous le testons avec les images de tests qui n'ont pas été donné auparavant.
Nous arrondissons les valeurs pour soit obtenir 1 (chien) ou 0 (chat).
Enfin, nous mettons les résultats dans un DataFrame pandas qui sera exporté en csv pour mieux visualiser les résultats.

In [ ]:
test_path = "./dataset/test1/"
X, y = create_test_data(test_path)
X_test, y = reshape_and_normalize_data(X, y)
predictions = model.predict(X_test)
predicted_val = [int(round(p[0])) for p in predictions]
submission_df = pd.DataFrame({'label':predicted_val})
submission_df.to_csv("submission.csv", index=True)